In [1]:
import sys
import tensorflow as tf
import numpy as np
import cv2,geocoder
import time
import datetime
from twilio.rest import Client
import yagmail
from geopy.geocoders import Nominatim

tf.compat.v1.disable_eager_execution()
import os



2023-03-18 08:06:04.084943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 08:06:04.218798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glob/development-tools/versions/oneapi/2023.0.1/oneapi/vpl/2023.0.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/tbb/2021.8.0/env/../lib/intel64/gcc4.8:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/rkcommon/1.10.0/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray_studio/0.11.1/lib:/glob/development-tools/versions/oneapi/2023.0.1/oneapi/ospray/2

In [26]:
def call(lat,lon):
  auth="b5b01672eee7ba30a0874c85a20baf29"
  account_sid = 'AC0b246b4a64109de691920c953996cded'
  client = Client(account_sid, auth)
  call = client.calls.create(twiml="<Response><Gather action=\"/gather_results\" digits=\"1\"><Say>Attention Required!! Garbage Detected... Garbage Detected... Garbage detected</Say></Gather></Response>",
    from_='+13853967299',
    to='+919392956575'
  )
  print(call.sid)

In [27]:
def sms(t1,lat,lon):
  auth = "b5b01672eee7ba30a0874c85a20baf29"
  sid = 'AC0b246b4a64109de691920c953996cded'
  geolocator = Nominatim(user_agent="geoapiExercises")
  location = geolocator.reverse(str(lat) + "," +str(lon))
  address = location.raw['address']
  google_maps_link = f"https://www.google.com/maps/search/?api=1&query={float(lat)},{float(lon)}"
  address = list(address.values())
  address = ",".join(address)
  case = "Attention Required!!"
  cl = Client(sid, auth)
  cl.messages.create(body=f"{case}\nScrap detected Time: {t1}\nLocation: {google_maps_link}\nAddress: {address}", from_ = '+13853967299', to = '+918897946277')


In [28]:
def email_generate(t1, lattitude, longitude):
  app_pass = "qirelusjswzhynjw"
  geolocator = Nominatim(user_agent="MyApp")
  #tval = t2 - t1
  Longitude = longitude
  Lattitude = lattitude
  location = geolocator.reverse(str(lattitude) + "," + str(Longitude))
  address = location.raw['address']
  google_maps_link = f"https://www.google.com/maps/search/?api=1&query={float(Lattitude)},{float(Longitude)}"
  from_address = "mgakhil04@gmail.com"#Provide your mail id
  to_address = "mgakhil03@gmail.com"#Provide receiver mail id
  subject = "Test Email"
  case = "Attention Required!!"
  address = list(address.values())
  address = ",".join(address)
  html = f"<h1>{case}</h1> <h2> Address: </h2><h3>{address}</h3><h3><a href={google_maps_link}>Location on GoogleMaps</a></h3><h2>Scrap detected Time:</h2><h3>{t1}</h3>"
  # Connect to your Gmail account and send the email
  yag = yagmail.SMTP(from_address,app_pass)
  contents=[html,yagmail.inline("./kill.jpg")]
  yag.send(to=to_address, subject=subject, contents=contents)
  print("Done")


In [29]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.compat.v1.GraphDef()
  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)
  return graph

In [30]:
def read_tensor_from_image_file(file_name, input_height=299, input_width=299,input_mean=0, input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.io.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(file_reader, channels = 3,name='png_reader')
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(tf.image.decode_gif(file_reader,name='gif_reader'))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
  else:
    image_reader = tf.image.decode_jpeg(file_reader, channels = 3,name='jpeg_reader')
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0);
  resized = tf.compat.v1.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.compat.v1.Session()
  result = sess.run(normalized)
  return result


def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.compat.v1.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

In [31]:
model_file = "files/retrained_graph.pb"
label_file = "files/retrained_labels.txt"
input_height = 224
input_width = 224
input_mean = 128
input_std = 128
input_layer = "input"
output_layer = "final_result"

In [32]:
def model(path):
  print("Running Model")
  graph = load_graph(model_file)
  
  video_reader = cv2.VideoCapture(path)
  li=[]

  try:
    c=2
    while True:
      ret, image_1 = video_reader.read()
      cv2.imwrite("kill1" + ".jpg", image_1)
      if c>1:
        cv2.imwrite("kill"+".jpg", image_1)
        c-=1
      file_name="kill1.jpg"
      for k in range(15):
       video_reader.grab()
      t = read_tensor_from_image_file(file_name,
                                      input_height=input_height,
                                      input_width=input_width,
                                      input_mean=input_mean,
                                      input_std=input_std)

      input_name = "import/" + input_layer
      output_name = "import/" + output_layer
      input_operation = graph.get_operation_by_name(input_name)
      output_operation = graph.get_operation_by_name(output_name)

      with tf.compat.v1.Session(graph=graph) as sess:
        start = time.time()
        results = sess.run(output_operation.outputs[0],
                          {input_operation.outputs[0]: t})
        end=time.time()
      results = np.squeeze(results)

      top_k = results.argsort()[-5:][::-1]
      labels = load_labels(label_file)
      #print(labels)
      print('\nEvaluation time (1-image): {:.3f}s\n'.format(end-start))
      template = "{} (score={:0.5f})"
      for i in top_k:
        print(template.format(labels[i], results[i]))
        if labels[i]=="not clean":
          li.append(results[i])
  except:
    pass
  print("Percentage that is not clean "+str(sum(li)*100/len(li)))
  density = sum(li)/len(li)
  g= geocoder.ip('me')
  lat, lon = g.latlng
  k=[lat,lon]
  geolocator = Nominatim(user_agent="geoapiExercises")
  location = geolocator.reverse(str(lat) + "," + str(lon))
  address = location.raw['address']
  address = list(address.values())
  address = ",".join(address)
  with open('emer.txt', 'w') as file:
    file.write(address)
  ct = datetime.datetime.now()
  end = time.time()
  print("Time Taken: ",end-start)
  
  if 0.80<density<0.96 :
    sms(ct,lat,lon)
  elif 0.96<density<0.98:
    whatsapp1(lat,lon)
  elif density>0.98:
    email_generate(ct,lat,lon)
  else:
   return "No Garbage Detected"
  return "Garbage Detected"


In [33]:
model("1 - Made with Clipchamp.mp4")

Running Model

Evaluation time (1-image): 0.137s

not clean (score=1.00000)
clean (score=0.00000)

Evaluation time (1-image): 0.148s

not clean (score=1.00000)
clean (score=0.00000)

Evaluation time (1-image): 0.146s

not clean (score=1.00000)
clean (score=0.00000)

Evaluation time (1-image): 0.134s

not clean (score=1.00000)
clean (score=0.00000)

Evaluation time (1-image): 0.138s

not clean (score=0.99994)
clean (score=0.00006)

Evaluation time (1-image): 0.133s

not clean (score=1.00000)
clean (score=0.00000)

Evaluation time (1-image): 0.142s

not clean (score=1.00000)
clean (score=0.00000)

Evaluation time (1-image): 0.142s

not clean (score=0.99973)
clean (score=0.00027)

Evaluation time (1-image): 0.131s

not clean (score=0.99994)
clean (score=0.00006)

Evaluation time (1-image): 0.144s

not clean (score=0.99139)
clean (score=0.00861)

Evaluation time (1-image): 0.134s

not clean (score=1.00000)
clean (score=0.00000)

Evaluation time (1-image): 0.145s

not clean (score=0.99982)


'Garbage Detected'